Tabla accounting_account_balances

In [5]:
# Asegúrate de que db_connection.py esté en la ruta correcta y contenga get_connection
from db_connection import get_connection

# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection() # Obtener una nueva conexión o reutilizar si get_connection maneja eso
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE accounting_account_balances;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'accounting_account_balances':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")



✅ Conexión exitosa a la base de datos
Columnas de la tabla 'accounting_account_balances':
  - id (bigint unsigned)
  - code (double(16,15))
  - accounting_id (bigint unsigned)
  - name (varchar(255))
  - initial_balance (double(18,6))
  - final_balance (double(18,6))
  - debit_movement (double(18,6))
  - credit_movement (double(18,6))
  - third_party_type_id (varchar(50))
  - third_party_id (bigint unsigned)
  - currency_id (varchar(255))
  - year (int)
  - month (int)
  - created_at (timestamp)
  - updated_at (timestamp)


Tabla accounting_accounts

In [8]:
# Asegúrate de que db_connection.py esté en la ruta correcta y contenga get_connection
from db_connection import get_connection

# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE accounting_accounts;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'accounting_accounts':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'accounting_accounts':
  - id (bigint unsigned)
  - name (varchar(250))
  - is_class (tinyint(1))
  - is_group (tinyint(1))
  - is_account (tinyint(1))
  - is_subaccount (tinyint(1))
  - is_auxiliary (tinyint(1))
  - is_subauxiliary (tinyint(1))
  - niif (tinyint(1))
  - cash_flow (tinyint(1))
  - exogenous (tinyint(1))
  - base_value (tinyint(1))
  - nature (varchar(1))
  - term (tinyint(1))
  - favorite (tinyint(1))
  - status (tinyint(1))
  - created_at (timestamp)
  - updated_at (timestamp)


Tabla accounting_movements

In [10]:
# Asegúrate de que db_connection.py esté en la ruta correcta y contenga get_connection
from db_connection import get_connection

# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE accounting_movements;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'accounting_movements':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'accounting_movements':
  - id (bigint unsigned)
  - class_id (bigint unsigned)
  - group_id (bigint unsigned)
  - account_id (bigint unsigned)
  - subaccount_id (bigint unsigned)
  - accounting_movement (bigint)
  - currency_id (varchar(255))
  - name (varchar(255))
  - date (timestamp)
  - debit_movement (double(18,6))
  - credit_movement (double(18,6))
  - third_party_type_id (varchar(50))
  - third_party_id (int unsigned)
  - document_type_id (smallint)
  - document (varchar(255))
  - document_id (bigint unsigned)
  - item_type (varchar(255))
  - item_id (int unsigned)
  - references_document_id (bigint unsigned)
  - payroll_employee_reference_id (int unsigned)
  - created_at (timestamp)
  - updated_at (timestamp)


Tabla accounting_voucher_items

In [11]:
# Asegúrate de que db_connection.py esté en la ruta correcta y contenga get_connection
from db_connection import get_connection

# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE accounting_voucher_items;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'accounting_voucher_items':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    # 3. Cerrar el cursor y la conexión de forma segura
    if 'cursor' in locals() and cursor is not None:
        cursor.close()
    if 'conexion' in locals() and conexion.is_connected():
        conexion.close()
    print("Conexión y cursor cerrados.")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'accounting_voucher_items':
  - id (bigint unsigned)
  - accounting_voucher_id (bigint unsigned)
  - third_party_type_id (varchar(255))
  - third_party_id (bigint unsigned)
  - accounting_account_id (bigint unsigned)
  - description (varchar(255))
  - debit_movement (double(18,6))
  - credit_movement (double(18,6))
  - created_at (timestamp)
  - updated_at (timestamp)
Conexión y cursor cerrados.


Tabla accounting_vouchers

In [12]:
# Asegúrate de que db_connection.py esté en la ruta correcta y contenga get_connection
from db_connection import get_connection

# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE accounting_vouchers;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'accounting_vouchers':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    # 3. Cerrar el cursor y la conexión de forma segura
    if 'cursor' in locals() and cursor is not None:
        cursor.close()
    if 'conexion' in locals() and conexion.is_connected():
        conexion.close()
    print("Conexión y cursor cerrados.")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'accounting_vouchers':
  - id (bigint unsigned)
  - accounting_voucher_type_id (bigint unsigned)
  - prefix (varchar(50))
  - number (bigint)
  - voucher_date (date)
  - third_party_id (bigint unsigned)
  - third_party_type_id (varchar(50))
  - advance_account_payable_id (bigint unsigned)
  - supplier_account_id (bigint unsigned)
  - advance_account_receivable_id (bigint unsigned)
  - cartera_account_id (bigint unsigned)
  - currency_id (varchar(255))
  - total_value (double(18,6))
  - voucher_status_id (smallint unsigned)
  - preferred (tinyint(1))
  - created_at (timestamp)
  - updated_at (timestamp)
Conexión y cursor cerrados.


Tabla accounting_voucher_types

In [13]:
# Asegúrate de que db_connection.py esté en la ruta correcta y contenga get_connection
from db_connection import get_connection

# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE accounting_voucher_types;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'accounting_voucher_types':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    # 3. Cerrar el cursor y la conexión de forma segura
    if 'cursor' in locals() and cursor is not None:
        cursor.close()
    if 'conexion' in locals() and conexion.is_connected():
        conexion.close()
    print("Conexión y cursor cerrados.")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'accounting_voucher_types':
  - id (bigint unsigned)
  - code (varchar(50))
  - name (varchar(60))
  - prefix (varchar(50))
  - initial_number (bigint)
  - current_number (bigint)
  - ledger_id (int unsigned)
  - status (tinyint(1))
  - created_at (timestamp)
  - updated_at (timestamp)
Conexión y cursor cerrados.


Tabla retention_concepts

In [14]:
# Asegúrate de que db_connection.py esté en la ruta correcta y contenga get_connection
from db_connection import get_connection

# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE retention_concepts;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'retention_concepts':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    # 3. Cerrar el cursor y la conexión de forma segura
    if 'cursor' in locals() and cursor is not None:
        cursor.close()
    if 'conexion' in locals() and conexion.is_connected():
        conexion.close()
    print("Conexión y cursor cerrados.")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'retention_concepts':
  - id (int unsigned)
  - description (text)
  - account_id (bigint unsigned)
  - created_at (timestamp)
  - updated_at (timestamp)
Conexión y cursor cerrados.


Tabla retentions

In [15]:
# Asegúrate de que db_connection.py esté en la ruta correcta y contenga get_connection
from db_connection import get_connection

# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE retentions;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'retentions':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    # 3. Cerrar el cursor y la conexión de forma segura
    if 'cursor' in locals() and cursor is not None:
        cursor.close()
    if 'conexion' in locals() and conexion.is_connected():
        conexion.close()
    print("Conexión y cursor cerrados.")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'retentions':
  - id (int unsigned)
  - retention_type_id (varchar(255))
  - retention_concept (int unsigned)
  - description (varchar(255))
  - percentage (double(10,5))
  - uvt_base (double(18,6))
  - base (double(18,6))
  - notes (varchar(255))
  - status (tinyint(1))
  - account_id (bigint unsigned)
  - user_id (int unsigned)
  - is_custom (varchar(255))
  - created_at (timestamp)
  - updated_at (timestamp)
Conexión y cursor cerrados.


Tabla retentions_applied

In [16]:
# Asegúrate de que db_connection.py esté en la ruta correcta y contenga get_connection
from db_connection import get_connection

# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE retentions_applied;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'retentions_applied':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    # 3. Cerrar el cursor y la conexión de forma segura
    if 'cursor' in locals() and cursor is not None:
        cursor.close()
    if 'conexion' in locals() and conexion.is_connected():
        conexion.close()
    print("Conexión y cursor cerrados.")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'retentions_applied':
  - id (bigint unsigned)
  - name (varchar(255))
  - type (varchar(255))
  - percentage (double(18,6))
  - base (double(18,6))
  - value (double(18,6))
  - retention_id (int unsigned)
  - contact_id (int unsigned)
  - document_id (bigint unsigned)
  - created_at (timestamp)
  - updated_at (timestamp)
Conexión y cursor cerrados.


Tabla retentions_certificates

In [17]:
# Asegúrate de que db_connection.py esté en la ruta correcta y contenga get_connection
from db_connection import get_connection

# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE retentions_certificates;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'retentions_certificates':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    # 3. Cerrar el cursor y la conexión de forma segura
    if 'cursor' in locals() and cursor is not None:
        cursor.close()
    if 'conexion' in locals() and conexion.is_connected():
        conexion.close()
    print("Conexión y cursor cerrados.")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'retentions_certificates':
  - id (bigint unsigned)
  - contact_id (int unsigned)
  - document_id (bigint unsigned)
  - value (double(18,6))
  - sent (tinyint(1))
  - certificate_url (varchar(255))
  - created_at (timestamp)
  - updated_at (timestamp)
Conexión y cursor cerrados.


Tabla taxes

In [18]:
# Asegúrate de que db_connection.py esté en la ruta correcta y contenga get_connection
from db_connection import get_connection

# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE taxes;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'taxes':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    # 3. Cerrar el cursor y la conexión de forma segura
    if 'cursor' in locals() and cursor is not None:
        cursor.close()
    if 'conexion' in locals() and conexion.is_connected():
        conexion.close()
    print("Conexión y cursor cerrados.")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'taxes':
  - id (smallint unsigned)
  - tax_type_id (varchar(255))
  - description (varchar(255))
  - percentage (double(10,5))
  - notes (varchar(255))
  - status (tinyint(1))
  - tax_account_deductible_id (bigint unsigned)
  - tax_account_generated_id (bigint unsigned)
  - user_id (int unsigned)
  - created_at (timestamp)
  - updated_at (timestamp)
Conexión y cursor cerrados.


Tabla payments

In [19]:
# Asegúrate de que db_connection.py esté en la ruta correcta y contenga get_connection
from db_connection import get_connection

# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection()
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE payments;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'payments':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    # 3. Cerrar el cursor y la conexión de forma segura
    if 'cursor' in locals() and cursor is not None:
        cursor.close()
    if 'conexion' in locals() and conexion.is_connected():
        conexion.close()
    print("Conexión y cursor cerrados.")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'payments':
  - id (bigint unsigned)
  - document_type_id (int unsigned)
  - billing_numbering_id (int unsigned)
  - payment_type (smallint unsigned)
  - contact_id (int unsigned)
  - number (int unsigned)
  - document_date (date)
  - document_hour (time)
  - associate_invoice (tinyint(1))
  - reference_document (json)
  - advance (tinyint(1))
  - subtotal (double(16,2))
  - discount (double(16,2))
  - gross (double(16,2))
  - pending (double(16,2))
  - total (double(16,2))
  - taxes (json)
  - retentions_value (double(16,2))
  - retentions (json)
  - retentions_details (json)
  - payments (json)
  - notes (varchar(255))
  - comments (varchar(255))
  - document_status_id (smallint unsigned)
  - DIAN_status (json)
  - contact_data (json)
  - details (json)
  - extra_data (json)
  - user_id (int unsigned)
  - deleted_at (timestamp)
  - created_at (timestamp)
  - updated_at (timestamp)
Conexión y cursor cerrados.
